<a href="https://colab.research.google.com/github/LeeEunSeo/NLP/blob/main/sentenceBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

ValueError: ignored

# SentenceBERT 모델 로드

In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510428 -0.7733841   0.592771   ...  0.57923526  0.32683456
  -0.6508967 ]
 [-0.09361677 -0.18191528 -0.19230838 ... -0.03165809  0.30412525
  -0.26793608]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

In [ ]:
df = pd.read_csv('wellness_dataset_original.csv')

df.head()

NameError: ignored

In [ ]:
df.info()

NameError: ignored

# 전처리

In [ ]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [ ]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [ ]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [ ]:
model.encode(df.loc[0, '유저'])

# 유저 대화내용 인코딩

In [ ]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806064, -0.29486933, 0.43790013, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.1450627, 0.2949032, -0.6739495..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520005, -0.081268415, 1.0945566, 0.10579..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679057, 0.46520728, 0.528507, -0.50760454..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277737, -0.37413877, 0.040532045, -0.862..."


In [ ]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [ ]:
text = '밖에 나가기 싫어'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 밖에 나가기 싫어
구분 증상/대인기피
챗봇 답변 : 충분히 그럴 수 있죠. 너무 불편하면 굳이 애써서 만날 필요도 없답니다.
유사도 0.47852277755737305


In [ ]:
text = '으아아아아아'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 으아아아아아
구분 감정/분노
챗봇 답변 : 화를 쏟아내고 나면 기분이 조금 풀릴 거예요.
유사도 0.6500303745269775


In [ ]:
text = '과제가 너무 많아'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 과제가 너무 많아
구분 감정/힘듦/스트레스
챗봇 답변 : 정말 힘들고 스트레스 받으시겠어요. 따뜻한 물에 샤워를 추천해드릴게요.
유사도 0.5377389192581177


In [ ]:
text = '혼자라 외로워'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 혼자라 외로워
구분 감정/외로움
챗봇 답변 : 저는 너무 외로울 때면 말랑한 무언가를 만지고는 해요. 단순한 행동에 집중하면 기분이 약간은 나아질 거예요.
유사도 0.7589062452316284


In [ ]:
text = '넌 누구야'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 넌 누구야
구분 감정/모호함
챗봇 답변 : 저라도 그런 감정을 느꼈을 거예요.
유사도 0.48895972967147827


In [ ]:
text = '인생이 재미없어'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 인생이 재미없어
구분 감정/부정적사고
챗봇 답변 : 부정적인 생각이 또 다른 부정적인 생각을 불러올 때가 있어요. 눈을 감고 숫자를 천천히 세보는 건 어때요?
유사도 0.6854720115661621


In [ ]:
text = '몰루'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 몰루
구분 배경/애완동물
챗봇 답변 : 그런 일이 있으셨군요.
유사도 0.2594057023525238


In [ ]:
text = '안녕하세요 저는 어쩌고 저쩌고 입니다.'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 안녕하세요 저는 어쩌고 저쩌고 입니다.
구분 배경/자각/정신질환
챗봇 답변 : 큰 이상이 아닐 수도 있어요. 꼭 전문가와 의논해보시길 바래요.
유사도 0.521621823310852


In [ ]:
text = '5글자 이상은 작성해야 유사도가 높네요'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 5글자 이상은 작성해야 유사도가 높네요
구분 배경/직장/양호
챗봇 답변 : 직장 생활에는 큰 문제가 없으시군요. 정말 다행입니다.
유사도 0.44624364376068115


In [ ]:
text = '요즘 살이 너무 쪘는데 어떡하죠'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 요즘 살이 너무 쪘는데 어떡하죠
구분 배경/건강문제/다이어트/스트레스
챗봇 답변 : 당신은 그대로도 멋진 사람이에요. 스트레스를 너무 많이 받는다면 무리하게 다이어트를 할 필요는 없어요.
유사도 0.6556819677352905


In [ ]:
text = '살이 안빠져요'

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])

질문 : 살이 안빠져요
구분 배경/건강문제/다이어트/스트레스
챗봇 답변 : 당신은 그대로도 멋진 사람이에요. 스트레스를 너무 많이 받는다면 무리하게 다이어트를 할 필요는 없어요.
유사도 0.6881242990493774


In [ ]:
text = ''

embedding = model.encode(text)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

answer = df.loc[df['distance'].idxmax()]
print('질문 :',text)
print('구분', answer['구분'])
#print('유사한 질문', answer['유저'])
print('챗봇 답변 :', answer['챗봇'])
print('유사도', answer['distance'])